In [ ]:
import cv2 as cv
import numpy as np
import os
import faiss
import insightface
import matplotlib.pyplot as plt

In [1]:
import numpy as np
import cv2 as cv
import os
import insightface
import pickle
from pathlib import Path
import matplotlib.pyplot as plt

class FACELOADING:
    """
    A class for loading and processing face images using InsightFace face detection.
    This class handles loading images from a directory structure, detecting faces,
    and storing their embeddings for face recognition.
    """
    
    def __init__(self, directory):
        """
        Initialize the FACELOADING class.
        
        Args:
            directory (str): Root directory containing subdirectories of face images
        """
        self.directory = directory
        self.X = []  # Will store face embeddings
        self.Y = []  # Will store corresponding labels
        # Initialize InsightFace buffalo_l model
        self.model = insightface.app.FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider'])
        self.model.prepare(ctx_id=0, det_size=(640, 640))

    def extract_face(self, filename):
        """
        Extract face embeddings from a single image using InsightFace.
        
        Args:
            filename (str): Path to the image file
            
        Returns:
            numpy.ndarray: Face embeddings or None if no face detected
        """
        try:
            # Read and convert image to RGB
            img = cv.imread(filename)
            if img is None:
                print(f"Could not read image: {filename}")
                return None
                
            img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            
            # Perform face detection and get embeddings
            faces = self.model.get(img)
            
            if not faces:
                print(f"No face detected in {filename}")
                return None
                
            # Get embeddings from the first detected face
            embeddings = faces[0].embedding
            # Normalize embeddings
            embeddings = embeddings / np.linalg.norm(embeddings)
            
            return embeddings
            
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
            return None

    def load_faces(self, dir):
        """
        Load all faces from a directory.
        
        Args:
            dir (str): Directory containing face images
            
        Returns:
            list: List of face embeddings
        """
        faces = []
        for im_name in os.listdir(dir):
            if im_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                path = os.path.join(dir, im_name)
                embeddings = self.extract_face(path)
                if embeddings is not None:
                    faces.append(embeddings)
        return faces

    def load_classes(self):
        """
        Load all classes (subjects) from the main directory.
        Each subdirectory name is treated as a class label.
        
        Returns:
            tuple: (numpy.ndarray of face embeddings, numpy.ndarray of labels)
        """
        self.X = []
        self.Y = []
        
        for sub_dir in os.listdir(self.directory):
            path = os.path.join(self.directory, sub_dir)
            if os.path.isdir(path):
                faces = self.load_faces(path)
                if faces:
                    self.X.extend(faces)
                    self.Y.extend([sub_dir] * len(faces))
                    print(f"Loaded {len(faces)} embeddings for subject: {sub_dir}")
        
        return np.array(self.X), np.array(self.Y)

/home/admin1/.pyenv/versions/3.10.16/lib/python3.10/site-packages/albumentations/check_version.py:107: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


In [2]:
face_loading=FACELOADING('Images')
X,Y=face_loading.load_classes()

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /home/admin1/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'],

/home/admin1/.pyenv/versions/3.10.16/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


Loaded 25 embeddings for subject: 7376232CT126
Loaded 25 embeddings for subject: 7376231CS310
Loaded 50 embeddings for subject: 7376231CS234
Loaded 25 embeddings for subject: 7376232AL156
Loaded 25 embeddings for subject: 7376231CD148
No face detected in Images/7376231CS328/DSC04960_aug3.JPG
No face detected in Images/7376231CS328/DSC04954_aug2.JPG
No face detected in Images/7376231CS328/DSC04950_aug2.JPG
Loaded 47 embeddings for subject: 7376231CS328
Loaded 50 embeddings for subject: 7376231CS325
Loaded 25 embeddings for subject: 7376232AL216
Loaded 25 embeddings for subject: 7376232AL107
Loaded 25 embeddings for subject: 7376241EI156
Loaded 25 embeddings for subject: 7376231CS130
Loaded 25 embeddings for subject: 7376232AL106
Loaded 25 embeddings for subject: 7376232AL143
Loaded 25 embeddings for subject: 7376232IT254
Loaded 25 embeddings for subject: 7376232AD189
Loaded 25 embeddings for subject: 7376231CS245
Loaded 25 embeddings for subject: 7376232AD195
Loaded 25 embeddings for su

In [3]:
X

array([[-0.00749791, -0.00039191, -0.03802159, ...,  0.01786169,
        -0.04637093, -0.08727191],
       [ 0.09549575,  0.02434655,  0.00731297, ...,  0.05545348,
         0.03798188, -0.04349744],
       [ 0.00924989, -0.03778453, -0.00129866, ..., -0.00708975,
        -0.09372527, -0.05204272],
       ...,
       [-0.00466251, -0.01907297,  0.05104661, ..., -0.00865108,
        -0.0850189 , -0.03333627],
       [ 0.00352195, -0.06367132,  0.05420497, ..., -0.03413382,
        -0.09022877, -0.05877259],
       [-0.01376407, -0.00175021,  0.04006452, ...,  0.00691231,
        -0.04810697, -0.03337606]], dtype=float32)

In [4]:
Y

array(['7376232CT126', '7376232CT126', '7376232CT126', '7376232CT126',
       '7376232CT126', '7376232CT126', '7376232CT126', '7376232CT126',
       '7376232CT126', '7376232CT126', '7376232CT126', '7376232CT126',
       '7376232CT126', '7376232CT126', '7376232CT126', '7376232CT126',
       '7376232CT126', '7376232CT126', '7376232CT126', '7376232CT126',
       '7376232CT126', '7376232CT126', '7376232CT126', '7376232CT126',
       '7376232CT126', '7376231CS310', '7376231CS310', '7376231CS310',
       '7376231CS310', '7376231CS310', '7376231CS310', '7376231CS310',
       '7376231CS310', '7376231CS310', '7376231CS310', '7376231CS310',
       '7376231CS310', '7376231CS310', '7376231CS310', '7376231CS310',
       '7376231CS310', '7376231CS310', '7376231CS310', '7376231CS310',
       '7376231CS310', '7376231CS310', '7376231CS310', '7376231CS310',
       '7376231CS310', '7376231CS310', '7376231CS234', '7376231CS234',
       '7376231CS234', '7376231CS234', '7376231CS234', '7376231CS234',
      

In [7]:
import faiss
# Setup FAISS IndexFlatIP
d = X.shape[1]
index = faiss.IndexFlatIP(d)

In [8]:
# Add data to FAISS index
if len(X) > 0:
    index.add(X)

In [9]:
faiss.write_index(index,'face_index.faiss')      

In [10]:
data={
    'labels':Y
}

In [11]:
with open('labels.pkl','wb') as f:
    pickle.dump(data,f)